In [10]:
import datetime
import numpy as np
import pandas as pd
from tqdm import tnrange
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

In [18]:
# imports needed and logging
import gzip
import gensim 
import logging
 
logging.basicConfig(format="%(asctime)s : %(levelname)s : %(message)s", level=logging.INFO)

## In this notebook, we'll construct a Item2Vec for our project

----

So that we can use these embedding to do more training!

In [3]:
# I'll use all of them for now. If filter, do it here
userLog = pd.read_csv("./data/userLog_201801_201802_for_participants.csv",sep=';')
userLog['datetime'] = pd.to_datetime(userLog['year'].astype(str)+"-"+ userLog['month'].astype(str) +"-"+ userLog['day'].astype(str)+" "+ userLog['hour'].astype(str)+":00:00")
userLog.drop(columns=['year','month','day','hour'], inplace=True)

In [11]:
proj2Idx = userLog.sort_values("project_id")['project_id'].unique()

getProjIdx = lambda x: np.where(proj2Idx==x)[0][0]
getProjIdx(4)

userLog['projectIdx'] = userLog['project_id'].apply(getProjIdx)

In [12]:
userLog1 = userLog.groupby("userCode").agg({"projectIdx":lambda x: list(x)})

In [13]:
userLog1.head()

,projectIdx
userCode,
00005aba-5ebc-0821-f5a9-bacca40be125,[2130]
0000bae7-6233-d7cc-2a6d-48aa70fe8ad4,[2383]
0000c576-e929-19eb-615a-349ec3b4709b,[2873]
0000d196-6385-80b8-661d-b7427042daa3,[4995]
0000e1e2-f595-0ae7-860f-fcc07dcb116e,"[3079, 3081]"


In [14]:
#moreThanOne = userLog2VecDf['project_id'].apply(len)>1
projs = userLog1['projectIdx']
projs = [[str(p) for p in proj] for proj in projs]
users = userLog1.index

In [85]:
documents = [TaggedDocument(proj, [user]) for i, (proj,user) in enumerate(zip(projs,users))]
documents

[TaggedDocument(words=['2130'], tags=['00005aba-5ebc-0821-f5a9-bacca40be125']),
 TaggedDocument(words=['2383'], tags=['0000bae7-6233-d7cc-2a6d-48aa70fe8ad4']),
 TaggedDocument(words=['2873'], tags=['0000c576-e929-19eb-615a-349ec3b4709b']),
 TaggedDocument(words=['4995'], tags=['0000d196-6385-80b8-661d-b7427042daa3']),
 TaggedDocument(words=['3079', '3081'], tags=['0000e1e2-f595-0ae7-860f-fcc07dcb116e']),
 TaggedDocument(words=['4813', '1733', '2971', '1733', '4813', '1387'], tags=['0000fa46-1f0b-9504-b568-43479d17620e']),
 TaggedDocument(words=['2850'], tags=['00011133-6093-88a6-98b8-de44b88170f3']),
 TaggedDocument(words=['1408'], tags=['00011607-0514-1c6c-9e93-30aca362f7e6']),
 TaggedDocument(words=['4234'], tags=['00015aac-868d-57da-d68a-1ff71b598380']),
 TaggedDocument(words=['2556', '4599', '5049'], tags=['00019a11-0aaa-2559-4feb-534724830d3e']),
 TaggedDocument(words=['2281'], tags=['0001eb13-0dbf-2a19-6a3f-893e17988350']),
 TaggedDocument(words=['2474'], tags=['0001f016-c997-6b0

In [89]:
model = Doc2Vec(documents, vector_size=300, window=3, min_count=2, workers=4,epochs=4,sample=1e-4, negative=10)

2018-07-19 22:28:53,137 : INFO : collecting all words and their counts
2018-07-19 22:28:53,138 : INFO : PROGRESS: at example #0, processed 0 words (0/s), 0 word types, 0 tags
2018-07-19 22:28:53,180 : INFO : PROGRESS: at example #10000, processed 24802 words (593172/s), 3766 word types, 10000 tags
2018-07-19 22:28:53,225 : INFO : PROGRESS: at example #20000, processed 48569 words (543854/s), 4330 word types, 20000 tags
2018-07-19 22:28:53,270 : INFO : PROGRESS: at example #30000, processed 73671 words (571342/s), 4589 word types, 30000 tags
2018-07-19 22:28:53,316 : INFO : PROGRESS: at example #40000, processed 97549 words (542851/s), 4724 word types, 40000 tags
2018-07-19 22:28:53,360 : INFO : PROGRESS: at example #50000, processed 121898 words (562355/s), 4809 word types, 50000 tags
2018-07-19 22:28:53,401 : INFO : PROGRESS: at example #60000, processed 146990 words (638868/s), 5009 word types, 60000 tags
2018-07-19 22:28:53,466 : INFO : PROGRESS: at example #70000, processed 172377 

2018-07-19 22:29:18,360 : INFO : EPOCH 1 - PROGRESS: at 41.09% examples, 48395 words/s, in_qsize 7, out_qsize 0
2018-07-19 22:29:19,614 : INFO : EPOCH 1 - PROGRESS: at 46.06% examples, 48869 words/s, in_qsize 7, out_qsize 0
2018-07-19 22:29:20,697 : INFO : EPOCH 1 - PROGRESS: at 50.27% examples, 49078 words/s, in_qsize 7, out_qsize 0
2018-07-19 22:29:21,900 : INFO : EPOCH 1 - PROGRESS: at 55.19% examples, 49593 words/s, in_qsize 8, out_qsize 0
2018-07-19 22:29:23,035 : INFO : EPOCH 1 - PROGRESS: at 59.28% examples, 49540 words/s, in_qsize 7, out_qsize 0
2018-07-19 22:29:24,215 : INFO : EPOCH 1 - PROGRESS: at 63.42% examples, 49393 words/s, in_qsize 7, out_qsize 0
2018-07-19 22:29:25,524 : INFO : EPOCH 1 - PROGRESS: at 68.45% examples, 49532 words/s, in_qsize 8, out_qsize 0
2018-07-19 22:29:26,701 : INFO : EPOCH 1 - PROGRESS: at 73.46% examples, 49973 words/s, in_qsize 8, out_qsize 0
2018-07-19 22:29:27,953 : INFO : EPOCH 1 - PROGRESS: at 78.58% examples, 50211 words/s, in_qsize 8, out_

2018-07-19 22:30:31,683 : INFO : EPOCH 4 - PROGRESS: at 20.37% examples, 48533 words/s, in_qsize 7, out_qsize 0
2018-07-19 22:30:32,773 : INFO : EPOCH 4 - PROGRESS: at 23.79% examples, 47399 words/s, in_qsize 8, out_qsize 0
2018-07-19 22:30:33,801 : INFO : EPOCH 4 - PROGRESS: at 28.58% examples, 49660 words/s, in_qsize 7, out_qsize 0
2018-07-19 22:30:34,889 : INFO : EPOCH 4 - PROGRESS: at 32.76% examples, 49905 words/s, in_qsize 7, out_qsize 0
2018-07-19 22:30:36,207 : INFO : EPOCH 4 - PROGRESS: at 36.86% examples, 48926 words/s, in_qsize 7, out_qsize 0
2018-07-19 22:30:37,424 : INFO : EPOCH 4 - PROGRESS: at 42.84% examples, 50600 words/s, in_qsize 7, out_qsize 0
2018-07-19 22:30:38,776 : INFO : EPOCH 4 - PROGRESS: at 46.91% examples, 49582 words/s, in_qsize 7, out_qsize 0
2018-07-19 22:30:40,179 : INFO : EPOCH 4 - PROGRESS: at 52.68% examples, 50176 words/s, in_qsize 7, out_qsize 0
2018-07-19 22:30:41,302 : INFO : EPOCH 4 - PROGRESS: at 56.79% examples, 50114 words/s, in_qsize 7, out_

### Let's see if it works or not

In [90]:
proj2Idx[0]

4

In [91]:
all_project = model.wv.vocab
def recommendByProjects(userId, userLog=userLog,topn = 7, withProb = False):
    
    seen = [str(i) for i in userLog[userLog['userCode']==userId].projectIdx.values]
    if len(seen) ==0: return "5858 3738 4201 5452 5678 5764 5067"
    most_similar_proj = model.wv.most_similar(positive=seen,topn=7+len(seen))
    recommending_projs = []
    if withProb: return most_similar_proj
    
    for proj, similarity in most_similar_proj:
        if proj not in seen: recommending_projs.append(proj)
        if len(recommending_projs)>=topn: break

    #     print(userLog.loc[user].project_id, similarity)
    
    
    return " ".join([str(proj2Idx[int(i)]) for i in recommending_projs])

recommendByProjects("7717bdc2-ea3e-e8ad-5d6b-178bd71c38b2", withProb =False)

2018-07-19 22:30:52,379 : INFO : precomputing L2-norms of word weight vectors
/home/users/u5722792744_1/miniconda3/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


'3054 3675 5721 6544 3546 4140 3462'

In [98]:
list_proj2Idx = list(proj2Idx)#.index(29)

## Test and evaluation

In [113]:
'1' in all_project

True

In [114]:
all_project = model.wv.vocab
def recommendByProjects(userId, userLog=userLog,topn = 7, withProb = False):
    
    seen = [str(i) for i in userLog[userLog['userCode']==userId].projectIdx.values]
    seen = [i for i in seen if i in all_project]
    if len(seen) ==0: return "5858 3738 4201 5452 5678 5764 5067"
    most_similar_proj = model.wv.most_similar(positive=seen,topn=7+len(seen))
    recommending_projs = []
    if withProb: return most_similar_proj
    
    for proj, similarity in most_similar_proj:
        if proj not in seen: recommending_projs.append(proj)
        if len(recommending_projs)>=topn: break

    #     print(userLog.loc[user].project_id, similarity)
    
    
    return " ".join([str(proj2Idx[int(i)]) for i in recommending_projs])

recommendByProjects("7717bdc2-ea3e-e8ad-5d6b-178bd71c38b2", withProb =False)

/home/users/u5722792744_1/miniconda3/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


'3054 3675 5721 6544 3546 4140 3462'

In [115]:
userLog.projectIdx.values

array([4063, 4063, 1987, ...,  304, 4717, 3919])

In [116]:
scores = []
for i in tnrange(5):
    testing_users = pd.read_csv("./clean_data/test_data/test_before_021718_700row_fold{}.csv".format(i))
    print(testing_users.shape)
    testing_users['project_id'] = testing_users['userCode'].apply(recommendByProjects)
    print("Finish grouping")
    testing_users['ans_project_id'] = testing_users['ans_project_id'].apply(eval)
    #testing_users['ans_project'] = testing_users['ans_project_id'].apply(lambda x: list(set(x)))
    print("Calculating AP")
    testing_users['MAP'] = testing_users.apply(lambda x: score_AP(x['ans_project_id'],x['project_id']),axis=1)
    score = testing_users['MAP'].mean()
    print(score)
    scores.append(score)
print(sum(scores)/len(scores))  #0.04469187

HBox(children=(IntProgress(value=0, max=5), HTML(value='')))

(700, 2)


/home/users/u5722792744_1/miniconda3/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


Exception in thread Thread-135:
Traceback (most recent call last):
  File "/home/users/u5722792744_1/miniconda3/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/home/users/u5722792744_1/miniconda3/lib/python3.6/site-packages/tqdm/_tqdm.py", line 148, in run
    for instance in self.tqdm_cls._instances:
  File "/home/users/u5722792744_1/miniconda3/lib/python3.6/_weakrefset.py", line 60, in __iter__
    for itemref in self.data:
RuntimeError: Set changed size during iteration



Finish grouping
Calculating AP
0.0
(700, 2)
Finish grouping
Calculating AP
0.0
(700, 2)
Finish grouping
Calculating AP
0.0
(700, 2)
Finish grouping
Calculating AP
0.0
(700, 2)
Finish grouping
Calculating AP
0.0

0.0


In [117]:
testing_users = pd.read_csv("./clean_data/test_data/test_before_021718_700row_fold{}.csv".format(i))
print(testing_users.shape)
testing_users['project_id'] = testing_users['userCode'].apply(recommendByProjects)
print("Finish grouping")
testing_users['ans_project_id'] = testing_users['ans_project_id'].apply(eval)
#testing_users['ans_project'] = testing_users['ans_project_id'].apply(lambda x: list(set(x)))
print("Calculating AP")
testing_users['MAP'] = testing_users.apply(lambda x: score_AP(x['ans_project_id'],x['project_id']),axis=1)
score = testing_users['MAP'].mean()
print(score)
scores.append(score)
print(sum(scores)/len(scores))  #0.04469187

(700, 2)


/home/users/u5722792744_1/miniconda3/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


Finish grouping
Calculating AP
0.0
0.0


In [124]:
 testing_users.head(10).apply(lambda x: score_AP(x['ans_project_id'],x['project_id']),axis=1)

['4885', '5635', '3838', '2332', '7996', '5203', '5182'] [4088]
['7235', '7193', '5046', '7338', '7618', '6865', '7361'] [8592, 7617, 6542]
['5720', '5723', '6902', '5617', '7305', '3644', '6287'] [7134]
['4660', '7308', '7119', '6839', '7262', '8712', '6170'] [5440, 6546]
['3108', '6129', '4834', '3984', '6536', '7035', '3775'] [3085]
['3717', '8223', '8317', '6524', '3298', '5099', '8841'] [8987]
['5906', '4828', '3838', '2891', '1828', '4399', '8833'] [4001, 4662]
['6520', '7963', '4669', '3603', '4653', '4863', '8322'] [3816, 4927]
['3525', '5235', '8957', '8893', '4930', '9318', '2754'] [8072, 2767]
['6928', '7194', '7174', '7200', '7166', '5118', '8883'] [6836]


0    0.0
1    0.0
2    0.0
3    0.0
4    0.0
5    0.0
6    0.0
7    0.0
8    0.0
9    0.0
dtype: float64

In [123]:
def score_AP(correct, recommended, n=7):
    recommended = recommended.split(" ")
    print(recommended, correct)
    assert(len(set(recommended))==len(recommended)) # Recommended should be unique    
    precisions = []
    correct_score = 1
    for idx, recommend_item in enumerate(recommended):
        try:
            if int(recommend_item) not in correct: precisions.append(0)
            else:                         
                precisions.append(correct_score/(idx+1))
                correct_score+=1
        except:
            print(recommend_item)
    return sum(precisions)/min(len(correct),n)
#score_AP(["5858", "5678"],"5678 5858 8058 8107 6527 9132 6446",7)

testing_users.head()

,userCode,ans_project_id,project_id,MAP
0,3eba491c-1735-110b-9a53-ea8370a3589e,[4088],4885 5635 3838 2332 7996 5203 5182,0.0
1,59acf503-9eee-7a4f-c85f-e118c103d16d,"[8592, 7617, 6542]",7235 7193 5046 7338 7618 6865 7361,0.0
2,444273a6-a620-c12e-04e1-ebca430427ca,[7134],5720 5723 6902 5617 7305 3644 6287,0.0
3,477e9529-ce7b-fb3a-165e-23f1bdc89a83,"[5440, 6546]",4660 7308 7119 6839 7262 8712 6170,0.0
4,5234c3d9-7111-7f88-091e-e3b05c1f1e53,[3085],3108 6129 4834 3984 6536 7035 3775,0.0


In [126]:
userLog.head()

,userCode,project_id,requestedDevice,userAgent,pageReferrer,datetime,projectIdx
0,7717bdc2-ea3e-e8ad-5d6b-178bd71c38b2,7956,Mobile,Android,HomeWebsite,2018-01-01,4063
1,7717bdc2-ea3e-e8ad-5d6b-178bd71c38b2,7956,Mobile,Android,HomeWebsite,2018-01-01,4063
2,cb5b4b68-cc01-6db6-f54b-4a0f881301c5,5067,Mobile,iPhone,HomeWebsite,2018-01-01,1987
3,5f74cef2-0d1e-b619-3564-0955a14e0985,6654,Mobile,iPhone,Google,2018-01-01,3033
4,dba8f279-844e-eef6-73ac-22bd7d1353cc,6474,Mobile,iPad,Google,2018-01-01,2884


### Use fitted item2vec embedding into the model

----

The difficult thing is that not all words are trained

In [1]:
modelwv = Doc2Vec.load("./model/doc2vec_before_train.wv")
vecs = modelwv.wv.syn0

NameError: name 'Doc2Vec' is not defined

In [295]:
# For projectId in vocab, 
    # Get the vec of that projectId
    # Convert projectId to projectIdx
    # in a new matrix, set the projectIdx rows to the vector
weights = np.random.randn(len(proj2Idx),300) 
weights.shape
# getProjIdx
for projId in modelwv.wv.vocab:

        itemVec = modelwv[projId]
        itemIdx = getProjIdx(int(projId))
        weights[itemIdx,:] = itemVec

In [296]:
weights.shape

(5395, 300)

In [298]:
np.savez(open("./data/item2vec2.weights","wb"),weights)

In [297]:
weights[5379:]

array([[ 0.00887639,  0.07015023,  0.00781579, ..., -0.01839984,
         0.01382158, -0.00368242],
       [-2.16615212, -0.52364787, -0.76826379, ...,  1.44584945,
         1.14934616,  0.73659924],
       [-1.64810962, -0.56298122, -0.51319717, ...,  0.78430585,
         0.21538076,  0.00908141],
       ...,
       [ 0.76432411, -1.52227833,  0.71535194, ...,  0.4098613 ,
         0.44062014, -1.08612642],
       [-0.88079915,  0.68700268, -0.47582725, ..., -0.25402968,
         0.00520405,  1.23822615],
       [-0.81227737,  0.46418154,  0.65941698, ..., -0.19681884,
         0.55788577, -0.86326219]])

In [280]:
len(modelwv.wv.vocab)

5317

In [278]:
pretrained_weights = np.load("./data/item2vec.weights")['arr_0']
pretrained_weights.shape

(5395, 300)

In [263]:
itemIdx

9420

In [220]:
vecs.shape
modelwv.wv.vocab

{'5342': <gensim.models.keyedvectors.Vocab at 0x7f68830425f8>,
 '5678': <gensim.models.keyedvectors.Vocab at 0x7f6882736320>,
 '6461': <gensim.models.keyedvectors.Vocab at 0x7f6882736160>,
 '9040': <gensim.models.keyedvectors.Vocab at 0x7f6882736d30>,
 '6709': <gensim.models.keyedvectors.Vocab at 0x7f68827367f0>,
 '6712': <gensim.models.keyedvectors.Vocab at 0x7f6882736a90>,
 '8829': <gensim.models.keyedvectors.Vocab at 0x7f6882736da0>,
 '4703': <gensim.models.keyedvectors.Vocab at 0x7f6882736780>,
 '6577': <gensim.models.keyedvectors.Vocab at 0x7f6882736ac8>,
 '4184': <gensim.models.keyedvectors.Vocab at 0x7f6882736748>,
 '6425': <gensim.models.keyedvectors.Vocab at 0x7f6882736400>,
 '4210': <gensim.models.keyedvectors.Vocab at 0x7f6882736048>,
 '8159': <gensim.models.keyedvectors.Vocab at 0x7f6882736dd8>,
 '8587': <gensim.models.keyedvectors.Vocab at 0x7f68827360b8>,
 '5985': <gensim.models.keyedvectors.Vocab at 0x7f68827369b0>,
 '9095': <gensim.models.keyedvectors.Vocab at 0x7f68827

## Predicting user

In [140]:

#userLog['datetime'] = pd.to_datetime(userLog['year'].astype(str)+"-"+ userLog['month'].astype(str) +"-"+ userLog['day'].astype(str)+" "+ userLog['hour'].astype(str)+":00:00")
#userLog.drop(columns=['year','month','day','hour'], inplace=True)

In [141]:
submission = pd.read_csv("./data/testing_users.csv")
userLog2 = userLog.sort_values("datetime")

In [142]:

userLogProjectId = userLog2.groupby("userCode").agg({"projectIdx":lambda x:list(x)})

submission_features = submission.merge(userLogProjectId, on='userCode')
submission_features.head()

,userCode,projectIdx
0,003366c6-0cd9-48e7-d134-2051d7360c2d,"[2570, 2570]"
1,003b2f18-60b4-37be-c63d-0caead1afa97,"[2552, 2552]"
2,003d571e-9de7-6de3-2cb6-2ae190cc7beb,"[3031, 3031, 3031, 3171]"
3,00426ede-6813-949b-de2e-6d8b228dff1d,[4281]
4,0043a9c7-cace-1aa1-3188-4b173bf28991,[4383]


In [143]:
submission_features['seq_len'] = submission_features['projectIdx'].apply(len)
maxSeq = submission_features['seq_len'].max()

In [168]:
submission_features = submission_features.sort_values('seq_len',ascending=False)

In [169]:
X_ans = submission_features['projectIdx'].values

In [170]:
X_ans2 = [row + [0]*(maxSeq- len(row)) for row in X_ans]
seq_ans2 = submission_features['seq_len']

In [171]:
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import torch.autograd as autograd
from torch.autograd import Variable
from torch.nn.utils.rnn import pad_packed_sequence, pack_padded_sequence

In [172]:
isCuda = False

if isCuda ==True:
    X_ans = Variable(torch.Tensor(X_ans2).long().cuda())
    seq_ans = torch.Tensor(seq_ans2.tolist()).long().cuda()
    
else:
    X_ans = Variable(torch.Tensor(X_ans2)).long()
    seq_ans = torch.Tensor(seq_ans2.tolist()).long()

In [173]:
project_num = 5395

In [300]:

hidden_size = 200
lstm_hidden_size = 50
embeddimg_dim = 300

class LSTM(nn.Module):
    def __init__(self):
        super(LSTM, self).__init__()
        
        
        self.use_last =True
        self.projEmbedding = nn.Embedding(5395, embeddimg_dim)#.from_pretrained(weight) # Spreading tuple into embedding dim
#         self.projEmbedding = from_pretrained(weights, False)
        
        self.lstm = nn.LSTM(embeddimg_dim,lstm_hidden_size, bidirectional = False)
        
        self.fc1 = nn.Linear(lstm_hidden_size, hidden_size)
        self.hidden2out = nn.Linear(hidden_size, project_num)
        
        #self.init_hidden()
    
    def init_hidden(self, batch_size):
        #TODO: change to clicking context
        if next(self.parameters()).is_cuda:
            self.hidden = (Variable(torch.zeros(1, batch_size, lstm_hidden_size).cuda()),Variable(torch.zeros(1, batch_size, lstm_hidden_size).cuda()))
        else:
            self.hidden = (Variable(torch.zeros(1, batch_size, lstm_hidden_size)),Variable(torch.zeros(1, batch_size, lstm_hidden_size)))
        
    def forward(self, inputs, lengths):

        batch_size = inputs.size()[0]
        
        embeds = self.projEmbedding(inputs)
        
        transposed_embeds = embeds.transpose(0,1)
        
        self.init_hidden(batch_size)
        
        if next(self.parameters()).is_cuda: lengths = lengths.cpu()
            
        packed_input = pack_padded_sequence(transposed_embeds, lengths.numpy())
        packed_outputs, (ht, ct) = self.lstm(packed_input, self.hidden)
        
        out_rnn, _ = pad_packed_sequence(packed_outputs)
        
        out_rnn = out_rnn.transpose(0,1)

        
        row_indices = torch.arange(0, batch_size).long()
        col_indices = lengths - 1
        if next(self.parameters()).is_cuda:
            row_indices = row_indices.cuda()
            col_indices = col_indices.cuda()
        
        
        if self.use_last:
            last_tensor=out_rnn[row_indices, col_indices, :]
        else:
            # use mean
            last_tensor = out_rnn[row_indices, :, :]
            last_tensor = torch.mean(last_tensor, dim=1)
        
        out = self.fc1(last_tensor)
        out = self.hidden2out(out)
        
        return out#out.transpose(0,1)#
    
        

In [301]:
model = LSTM()
model.load_state_dict(torch.load("./model/LSTM_v2.weight"))

In [302]:
preds_output = model(X_ans,seq_ans)

In [303]:
val, pred_idx = torch.topk(preds_output, 30)
predictions = pred_idx.data

In [304]:
proj2Idx = userLog.sort_values("project_id")['project_id'].unique()
getProjIdx = lambda x: np.where(proj2Idx==x)[0][0]
getProjIdx(4)

0

In [305]:
seen_answer = submission_features['projectIdx'].values

preds = []
for pred, seen  in tqdm(zip(predictions, seen_answer),total=5):
    #print(pred,ans,seen)
    # Calculate here
    selected_pred = []
    for p in pred:
        if p not in seen:
            
            selected_pred.append(str(proj2Idx[p]))
        if len(selected_pred)==7: 
            preds.append(" ".join([i for i in selected_pred]))
            break
    

7408it [00:00, 29147.99it/s]         


In [195]:
from tqdm import tqdm

In [309]:
submission_features['project_id'] = preds
submission_features.head()

,userCode,projectIdx,seq_len,project_id
4272,24a0d17a-5df6-e68e-de6c-e668551e82b6,"[2369, 2505, 2369, 2369, 508, 4214, 4214, 4214...",55,9483 3738 5858 9106 6425 4201 5263
6524,c3d31d2f-b147-6313-1e1a-189c92f01db0,"[3780, 4466, 2794, 2794, 4399, 4654, 4170, 415...",51,9483 6511 5858 3738 9106 5452 6425
3805,02cae1e4-f73f-958e-03cd-3ca3b04b451d,"[2851, 2289, 2289, 3167, 253, 253, 3167, 3167,...",48,9483 5858 6511 3738 9106 6425 5452
6257,b118a055-e891-1b24-8751-bc36fd89a175,"[587, 587, 587, 587, 587, 5066, 587, 587, 5052...",47,9483 5858 6511 9106 3738 6425 5452
6642,cbf8d800-6088-659c-df8c-88ec53c17dc6,"[2725, 2725, 544, 544, 544, 544, 544, 4808, 54...",45,9483 5858 6511 3738 5452 9106 6425


In [310]:
submission_features[['userCode','project_id']].to_csv("./submission/deeplstm_v2.csv",index=False)

In [211]:
submission_features.head()

,userCode,projectIdx,seq_len,project_id
4272,24a0d17a-5df6-e68e-de6c-e668551e82b6,"[2369, 2505, 2369, 2369, 508, 4214, 4214, 4214...",55,5858 9483 3738 6425 9106 4201 5263
6524,c3d31d2f-b147-6313-1e1a-189c92f01db0,"[3780, 4466, 2794, 2794, 4399, 4654, 4170, 415...",51,5858 6511 3738 9483 9106 5452 6425
3805,02cae1e4-f73f-958e-03cd-3ca3b04b451d,"[2851, 2289, 2289, 3167, 253, 253, 3167, 3167,...",48,5858 9483 6511 3738 9106 6425 4201
6257,b118a055-e891-1b24-8751-bc36fd89a175,"[587, 587, 587, 587, 587, 5066, 587, 587, 5052...",47,5858 6511 3738 9483 5452 5678 6425
6642,cbf8d800-6088-659c-df8c-88ec53c17dc6,"[2725, 2725, 544, 544, 544, 544, 544, 4808, 54...",45,5858 9483 6511 3738 9106 6425 5452
